In [1]:
import pprint

import ee
# import geemap

import openet.disalexi

ee.Initialize()

In [2]:
# Washington image
image_id = 'LANDSAT/LC08/C02/T1_L2/LC08_043028_20220808' 
good_xy = [-117.48, 46.32]
bad_xy = [-117.56, 46.28]


# image_id = 'LANDSAT/LC09/C02/T1_L2/LC09_032032_20220702'
# good_xy = [-102.84, 40.72]
# bad_xy = [-102.84, 40.68]

landsat_img = ee.Image(image_id)
image_dt = ee.Date(landsat_img.get('system:time_start')).update(hour=0, minute=0, second=0)

In [3]:
# ALEXI stuff
alexi_coll_id = 'projects/ee-tulipyangyun-2/assets/alexi/ALEXI_V006'
alexi_mask_id = 'projects/earthengine-legacy/assets/projects/disalexi/alexi/conus_v004_mask'
alexi_mask = ee.Image(alexi_mask_id).double().multiply(0)
alexi_geo = [0.04, 0.0, -125.02, 0.0, -0.04, 49.78]
alexi_cs = 0.04
alexi_x, alexi_y = -125.02, 49.78
alexi_crs = 'EPSG:4326'

In [4]:
def point_image_value(image, xy, scale=1000):
    """Extract the output value from a calculation at a point"""
    return (
        ee.Image(image)
        .reduceRegion(
            reducer=ee.Reducer.first(), geometry=ee.Geometry.Point(xy), 
            crs=alexi_crs, crsTransform=alexi_geo,
            # scale=scale,
        )
        .getInfo()
    )

In [5]:
disalexi_obj = openet.disalexi.Image.from_image_id(
    image_id,
    alexi_source = 'projects/ee-tulipyangyun-2/assets/alexi/ALEXI_V006',
    lai_source = 'openet-landsat-lai',
    tir_source = 'projects/openet/assets/lst/landsat/c02',
    elevation_source = 'USGS/SRTMGL1_003',
    landcover_source = 'USGS/NLCD_RELEASES/2019_REL/NLCD',
    air_pres_source = 'CFSR',
    air_temp_source = 'CFSR',
    rs_daily_source = 'CFSR',
    rs_hourly_source = 'CFSR',
    vapor_pre_source = 'CFSR',
    wind_speed_source = 'CFSR',
    stability_iterations = 10,
    albedo_iterations = 10,
    et_min = 0,
)
# print(disalexi_obj)

In [6]:
# Check the Ta initial (direct) values computed from the meteorology
# Why is the direct value so low for the bad point?
ta_init_img = disalexi_obj.ta_coarse_initial()
pprint.pprint(point_image_value(ta_init_img, good_xy))
pprint.pprint(point_image_value(ta_init_img, bad_xy))


{'ta_initial': 289}
{'ta_initial': 268}


In [7]:
# Generate a mosaic of Ta values from the initial image
ta_new_mosaic_img = disalexi_obj.ta_mosaic(
    ta_init_img, 
    ta_offsets=[-20, -12, -7, -4, -2, -1, 0, 1, 2, 4, 7, 12, 20]
)
pprint.pprint(point_image_value(ta_new_mosaic_img, good_xy))
pprint.pprint(point_image_value(ta_new_mosaic_img, bad_xy))


{'step_00_bias': -2.694891502435842,
 'step_00_ta': 269,
 'step_01_bias': -2.616300575534045,
 'step_01_ta': 277,
 'step_02_bias': -2.4743902468297536,
 'step_02_ta': 282,
 'step_03_bias': -1.9756256142209974,
 'step_03_ta': 285,
 'step_04_bias': -1.5006476815464556,
 'step_04_ta': 287,
 'step_05_bias': -1.2440787243142921,
 'step_05_ta': 288,
 'step_06_bias': -0.9605537860193898,
 'step_06_ta': 289,
 'step_07_bias': -0.6792622869704026,
 'step_07_ta': 290,
 'step_08_bias': -0.41291483070050816,
 'step_08_ta': 291,
 'step_09_bias': 0.10424604231675838,
 'step_09_ta': 293,
 'step_10_bias': 0.9495330391682941,
 'step_10_ta': 296,
 'step_11_bias': 2.3810347092022917,
 'step_11_ta': 301,
 'step_12_bias': 3.8915638468334657,
 'step_12_ta': 309}
{'step_00_bias': -2.7481478087062747,
 'step_00_ta': 248,
 'step_01_bias': -2.7471607765800465,
 'step_01_ta': 256,
 'step_02_bias': -2.6986033445601563,
 'step_02_ta': 261,
 'step_03_bias': -2.6278144066762334,
 'step_03_ta': 264,
 'step_04_bias': -

In [8]:
# Check if the ta_mosaic_interpolate() function works on the new mosaic image
ta_new_interp_img = openet.disalexi.ta_mosaic_interpolate(ta_new_mosaic_img)
pprint.pprint(point_image_value(ta_new_interp_img, good_xy))
pprint.pprint(point_image_value(ta_new_interp_img, bad_xy))


{'bias_a': -0.41291483070050816,
 'bias_b': 0.10424604231675838,
 'ta_a': 291,
 'ta_b': 293,
 'ta_interp': 292.6}
{'bias_a': -2.075045156911635,
 'bias_b': 0.02325412019285622,
 'ta_a': 280,
 'ta_b': 288,
 'ta_interp': 287.9}


In [9]:
# Try calling the ta_coarse function directly
ta_coarse_img = disalexi_obj.ta_coarse()
pprint.pprint(point_image_value(ta_coarse_img, good_xy))
pprint.pprint(point_image_value(ta_coarse_img, bad_xy))


{'bias_a': -0.41291483070050816,
 'bias_b': 0.10424604231675838,
 'ta_a': 291,
 'ta_b': 293,
 'ta_initial': 289,
 'ta_interp': 292.6}
{'bias_a': -2.075045156911635,
 'bias_b': 0.02325412019285622,
 'ta_a': 280,
 'ta_b': 288,
 'ta_initial': 268,
 'ta_interp': 287.9}


In [10]:
# Check the original Ta 10k values
ta_10k_coll_id = 'projects/openet/assets/disalexi/tair/conus_v006_10k'
ta_10k_coll = ee.ImageCollection(ta_10k_coll_id)
ta_10k_img = ta_10k_coll.filterMetadata('image_id', 'equals', image_id).first()
pprint.pprint(point_image_value(ta_10k_img, good_xy))
pprint.pprint(point_image_value(ta_10k_img, bad_xy))


{'step_0_bias': -2.772726993560791,
 'step_0_ta': 253,
 'step_10_bias': 6.795235687099801,
 'step_10_ta': 353,
 'step_1_bias': -2.7471981149715003,
 'step_1_ta': 263,
 'step_2_bias': -2.659191618786192,
 'step_2_ta': 273,
 'step_3_bias': -2.4135402624952227,
 'step_3_ta': 283,
 'step_4_bias': -0.01590148354443599,
 'step_4_ta': 293,
 'step_5_bias': 2.708105777565797,
 'step_5_ta': 303,
 'step_6_bias': 4.555698124452835,
 'step_6_ta': 313,
 'step_7_bias': 5.7840133920186485,
 'step_7_ta': 323,
 'step_8_bias': 6.268222685163138,
 'step_8_ta': 333,
 'step_9_bias': 6.543105732086612,
 'step_9_ta': 343}
{'step_0_bias': -2.7487878646850588,
 'step_0_ta': 252,
 'step_10_bias': 7.704137837595062,
 'step_10_ta': 352,
 'step_1_bias': -2.6806368854094056,
 'step_1_ta': 262,
 'step_2_bias': -2.4468619537934555,
 'step_2_ta': 272,
 'step_3_bias': -1.8609173427616161,
 'step_3_ta': 282,
 'step_4_bias': 1.1440170352924244,
 'step_4_ta': 292,
 'step_5_bias': 3.847881442367372,
 'step_5_ta': 302,
 'ste

In [11]:
# Check if the ta_mosaic_min_bias() function works on the existing 10k image
ta_min_bias_img = openet.disalexi.ta_mosaic_min_bias(ta_10k_img)
pprint.pprint(point_image_value(ta_min_bias_img, good_xy))
pprint.pprint(point_image_value(ta_min_bias_img, bad_xy))


{'ta': 293}
{'ta': 292}


In [12]:
# Check if the ta_mosaic_interpolate() function works on the existing 10k image
ta_10k_interp_img = openet.disalexi.ta_mosaic_interpolate(ta_10k_img)
pprint.pprint(point_image_value(ta_10k_interp_img, good_xy))
pprint.pprint(point_image_value(ta_10k_interp_img, bad_xy))


{'bias_a': -0.01590148354443599,
 'bias_b': 2.708105777565797,
 'ta_a': 293,
 'ta_b': 303,
 'ta_interp': 293.1}
{'bias_a': -1.8609173427616161,
 'bias_b': 1.1440170352924244,
 'ta_a': 282,
 'ta_b': 292,
 'ta_interp': 288.2}


In [13]:
# Check the meteorology values
ta_nldas_coll = (
    ee.ImageCollection('NASA/NLDAS/FORA0125_H002')
    .filterDate(image_dt, image_dt.advance(1, 'day'))
    .select(['temperature'])
)
# Pulling maximum air temperature instead of 0 UTC
ta_nldas_img = ee.Image(ta_nldas_coll.reduce(ee.Reducer.max())).add(273.15).floor()
ta_nldas_img = alexi_mask.add(ta_nldas_img).rename(['ta'])
print(point_image_value(ta_nldas_img, good_xy))
print(point_image_value(ta_nldas_img, bad_xy))


ta_source_coll_id = 'projects/disalexi/meteo_data/airtemperature/global_v001_3hour'
ta_source_coll = ee.ImageCollection(ta_source_coll_id).select(['temperature'])
ta_source_img = openet.disalexi.utils.interpolate(ta_source_coll, image_dt, timestep=3)
ta_source_img = (
    ee.Image(ta_source_img)
    .resample('bicubic').reproject(crs=alexi_crs, crsTransform=alexi_geo)
    #.resample('bilinear').reproject(crs=self.crs, crsTransform=self.transform)
)
print(point_image_value(ta_source_img, good_xy))
print(point_image_value(ta_source_img, bad_xy))


# ta_new_10k_img = disalexi_obj.ta_mosaic(ta_source_img, step_size=10, step_count=10)
# pprint.pprint(point_image_value(ta_new_10k_img, good_xy))
# pprint.pprint(point_image_value(ta_new_10k_img, bad_xy))


{'ta': 303}
{'ta': 302}
{'temperature': 303.1863216802033}
{'temperature': 302.99830490086805}


In [14]:
# # Check if the ta_mosaic_interpolate() function works on the existing 1k image
# ta_10k_interp_img = openet.disalexi.ta_mosaic_interpolate(ta_new_10k_img)

# pprint.pprint(point_image_value(ta_10k_interp_img, good_xy))
# pprint.pprint(point_image_value(ta_10k_interp_img, bad_xy))
